In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%load_ext autotime

# Sample Submission

In [2]:
#sample = pd.read_csv("sample_submission.csv")
#sample[:3]

time: 148 µs


In [3]:
#sample.prediction.value_counts()

time: 183 µs


# Test

In [4]:
#test_orig = pd.read_csv("test.csv")
#test_orig[:3]

time: 156 µs


# Train Cleaning

In [5]:
train_orig = pd.read_csv("train.csv")

time: 8.37 s


In [6]:
#y_train = train_orig['target'].copy()
#x_train = train_orig.loc[:, train_orig.columns != 'target'].copy()
#x_train = train_orig.loc[:, ~train_orig.columns.isin(['target','severe_toxicity','obscene',
#                                                    'identity_attack','insult','threat'])].copy()

time: 161 µs


In [7]:
train_orig[:2]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.0,"This is so cool. It's like, 'would you want yo...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.0,Thank you!! This would make my life a lot less...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4


time: 22 ms


In [8]:
# toxic comments
train_orig[train_orig.target >= 0.5][:2]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:50.865549+00,2,NaN,2006,rejected,0,0,0,0,0,0.009524,0,105


time: 91.5 ms


In [9]:
#x_train[:2]

time: 168 µs


# Fill NaNs with Zeros

In [10]:
#x_train.fillna(0, inplace=True)

time: 367 µs


In [11]:
train_orig.fillna(0, inplace=True)

time: 454 ms


# describe/summarize

In [12]:
desc_df = train_orig.copy()
desc_df.loc[desc_df.target >= 0.5, "toxic"] = True
desc_df.toxic.fillna(False,inplace=True)

time: 989 ms


In [13]:
rel_col = list(set(desc_df.columns.tolist()) - set(['target','id']))
desc_df[rel_col].groupby("toxic").describe()

publication_id                                                     other_gender                                                    asian                                                  threat                                                   other_religion                                              other_race_or_ethnicity                                              identity_attack                                                        other_disability                                              intellectual_or_learning_disability                                                   article_id                                                                                 buddhist                                              psychiatric_or_mental_illness                                                  insult                                                        identity_annotator_count                                                  physical_disability                      \
               count       mean        std  min   25%   50%   75%    max        count      mean       std  min  25%  50%  75%   max      count      mean       std  min  25%  50%  75%  max      count      mean       std  min  25%  50%       75%  max          count      mean       std  min  25%  50%  75%  max                   count      mean       std  min  25%  50%  75%  max           count      mean       std  min  25%       50%       75%  max            count      mean       std  min  25%  50%  75%  max                               count      mean       std  min  25%  50%  75%       max      count           mean            std     min       25%       50%       75%       max      count      mean       std  min  25%  50%  75%  max                         count      mean       std  min  25%  50%  75%  max      count      mean       std  min       25%       50%  75%  max                    count      mean        std  min  25%  50%  75%     max               count      mean       std   
toxic                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
False      1660540.0  49.728305  27.510128  2.0  21.0  54.0  54.0  115.0    1660540.0  0.000183  0.005397  0.0  0.0  0.0  0.0  0.75  1660540.0  0.002576  0.040615  0.0  0.0  0.0  0.0  1.0  1660540.0  0.006168  0.033738  0.0  0.0  0.0  0.000000  0.6      1660540.0  0.001443  0.017505  0.0  0.0  0.0  0.0  1.0               1660540.0  0.001718  0.019683  0.0  0.0  0.0  0.0  1.0       1660540.0  0.014580  0.054317  0.0  0.0  0.000000  0.000000  0.8        1660540.0  0.000260  0.006527  0.0  0.0  0.0  0.0  0.6                           1660540.0  0.000192  0.007026  0.0  0.0  0.0  0.0  1.000000  1660540.0  281220.733885  103935.348379  2006.0  160110.0  331961.0  366012.0  399541.0  1660540.0  0.000305  0.014429  0.0  0.0  0.0  0.0  1.0                     1660540.0  0.002314  0.039499  0.0  0.0  0.0  0.0  1.0  1660540.0  0.039851  0.091333  0.0  0.000000  0.000000  0.0  0.7                1660540.0  1.351100  16.231766  0.0  0.0  0.0  0.0  1860.0           1660540.0  0.000287  0.008115   
True  

time: 5.36 s


# lower case, stopwords, puncutation

In [14]:
#import nltk
#nltk.download('all')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')

time: 168 µs


In [15]:
pd.options.display.max_colwidth = 1000
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
import string

time: 610 ms


In [16]:
english_stopwords = set(stopwords.words('english'))
w_tokenizer = WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

time: 2.41 ms


In [17]:
def clean_message(msg):
    msg = msg.lower()
    msg_tokens = nltk.word_tokenize(msg)
    clean_msg_tokens = [w for w in msg_tokens if w not in english_stopwords]
    clean_msg_tokens_puct = [w for w in clean_msg_tokens if w not in string.punctuation]
    lemmatized_token = [lemmatizer.lemmatize(w) for w in clean_msg_tokens_puct]
    return lemmatized_token

time: 655 µs


In [18]:
# poorn
#train_orig['clean'] = train_orig.comment_text.apply(clean_message)

time: 145 µs


In [19]:
train_orig["clean"] = train_orig.comment_text.apply(lambda x: " ".join([words.strip(string.punctuation).lower()
for words in [word for word in x.split() if (word not in string.punctuation) & (word not in english_stopwords)]]))
train_orig["clean"] = train_orig["clean"].apply(lambda x: " ".join([lemmatizer.lemmatize(w) 
                                                                    for w in w_tokenizer.tokenize(x)]))

time: 3min 13s


In [20]:
train_orig['clean'][:2]

0          this cool it's like would want mother read this really great idea well done
1    thank you this would make life lot le anxiety-inducing keep up let anyone get way
Name: clean, dtype: object

time: 1.86 ms


# vectorize

In [21]:
from collections import Counter
import numpy as np

time: 7.06 ms


In [22]:
# pooran
#voc_counter = Counter(messages_df.clean_message.sum())

time: 146 µs


In [23]:
voc_counter = Counter(" ".join(train_orig.clean).split(" "))

time: 10.1 s


In [24]:
#for w,c in voc_counter.items():
#    print(w,c)

time: 7.32 ms


In [25]:
voc_set = dict((k, v) for k, v in voc_counter.items() if v > 10)
voc_dict = {w:i for w,i in zip(voc_set, range(len(voc_set)))}

time: 71.7 ms


In [26]:
train_orig["clean"] = train_orig["clean"].apply(lambda x: x.split())
train_orig.loc[train_orig.target >= 0.5, "toxic"] = True
train_orig.toxic.fillna(False,inplace=True)

time: 7.12 s


In [34]:
train_orig_sample = train_orig.sample(10000)

time: 62.6 ms


In [35]:
X = np.zeros((len(train_orig_sample), len(voc_set)), dtype = np.int8 )
for msg_idx, msg in enumerate(train_orig_sample['clean']):
    for word in set(msg):
        if word in voc_dict:
            X[msg_idx][voc_dict[word]] = msg.count(word)

time: 340 ms


In [36]:
y = np.where(train_orig_sample.toxic == True, 1, 0)

time: 638 µs


# Save

In [37]:
X.shape

(10000, 63574)

time: 1.01 ms


In [38]:
np.savetxt('y.txt', y, fmt='%d')

time: 10.8 ms


In [39]:
np.savetxt('X.txt', X, fmt='%d')

time: 1min 29s


# !!! restart kernel !!!

# Load

In [3]:
import numpy as np
X = np.loadtxt('X.txt', dtype=int)
y = np.loadtxt('y.txt', dtype=int)

# Model

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

In [5]:
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=.2,random_state=2)

In [6]:
clf = LogisticRegressionCV(cv=5,max_iter=200)
clf.fit(train_X,train_y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=200, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [7]:
clf.score(test_X,test_y)

0.9285